In [9]:
# copyright ############################### #
# This file is part of the Xtrack Package.  #
# Copyright (c) CERN, 2021.                 #
# ######################################### #


import xtrack as xt
import xobjects as xo
import xpart as xp

import ducktrack as dtk
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
from tqdm import tqdm








# beta_rel = 0.305
beta_rel = 0.106

# gamma = 1.050
gamma = 1.006


I=0.4
L = 1.5 # m cooler length
r_beam=25*1e-3

mass0=938.27208816*1e6 #ev/c^2

T_perp = 0.01 # <E> [eV] = kb*T
T_l =  0.001 # <E> [eV]
B = 0.060 # T for LEIR
Z=1


c=299792458.0

p0c = mass0*beta_rel*gamma #eV/c



length = 182.43280000000 #m
T = length/(c*beta_rel)
ms_per_turn = 1000*T





beta_x=10 
beta_y=4
emittance=10*1e-6

arc=dtk.LinearTransferMatrix(Q_x=5.38518956965, Q_y=5.36921653748,
                            beta_x_0=beta_x, beta_x_1=beta_x,
                            beta_y_0=beta_y,  beta_y_1=beta_y,
                            alpha_x_0=0,   alpha_x_1=0,
                            alpha_y_0=0,   alpha_y_1=0,
                            disp_x_0=0,disp_x_1=0,
                            disp_y_0=0,    disp_y_1=0,
                            beta_s=1*1e40,
                            Q_s=0,
                            chroma_x=0.0, chroma_y=0
                                           
                 )




num_particles=int(1e4)

#x=np.random.normal(0,np.sqrt(5*emittance) ,num_particles),
#px=np.random.normal(0, np.sqrt(emittance/5), num_particles),
#x=np.linspace(1*1e-3, 20*1e-3, num_particles),
#px=np.random.uniform(0, 0, num_particles),

dtk_particle = dtk.TestParticles(
        
        mass0=mass0,
        p0c=p0c,
        x=np.random.normal(0,np.sqrt(beta_x*emittance) ,num_particles),
        px=np.random.normal(0, np.sqrt(emittance/beta_x), num_particles),
        y=0,
        py=0,
        delta=0,
        zeta=0)

dtk_particle_copy_old=dtk_particle.copy()






In [10]:
import numpy as np
from tqdm import tqdm
from scipy.optimize import minimize_scalar

num_turns = 1

x = []
px = []
action = []
force = []
force_old = []


force_diff = []
def cost_function(b_ratio):
     

    xp_cooler=1e-3

    B_ratio=1e-3


    dtk_cooler = dtk.elements.ElectronCooler(I=I,L=L,r_beam=r_beam,
                                            T_perp=T_perp,T_l=T_l,
                                            B=B,Z=Z,B_ratio=B_ratio,
                                            Neutralisation_space_charge=0,
                                            xp_cooler=0)

    coolers = []
    num_coolers=int(1e4)

    for i in range(num_coolers):
        angle=np.random.normal(0,b_ratio,1)
        #angle=np.random.uniform(-B_ratio,B_ratio,1)
        #print('angle:',angle)
        cooler = dtk.elements.ElectronCooler(I=I, L=L, r_beam=r_beam,
                                            T_perp=T_perp, T_l=T_l,
                                            B=B, Z=Z, B_ratio=0,
                                            Neutralisation_space_charge=0,
                                            xp_cooler=angle)
        coolers.append(cooler)




    force_old = dtk_cooler.force(dtk_particle)



    for i in range(num_turns):
        x.append(dtk_particle.x)
        px.append(dtk_particle.px)

        action_temp = 0.5*(dtk_particle.x**2/beta_x + beta_x*dtk_particle.px**2) #Joule sec
        action.append(action_temp)

        force_temp=[]
        for cooler in coolers:
            f1=cooler.force(dtk_particle)
            force_temp.append(f1)

        force_temp=np.array(force_temp)
        force=force_temp.mean(axis=0)

        force_diff.append(np.abs(force - force_old))
        res=np.mean(force_diff)
        print('res',res)
    return res

# use scipy's minimize_scalar function to find the optimal b_ratio
with tqdm(total=1000) as pbar:
    def update_pbar(x):
        pbar.update()
    
    iter_result=0.0010000029046089275  
    iter_result=0.0010050858607870458  
    res = minimize_scalar(cost_function, method='bounded',tol=1e-10,bounds=(iter_result,1e-2),
                          options={'maxiter': 1000, 'disp': True})
optimal_b_ratio = res.x

# print the result
print(f"Optimal b_ratio: {optimal_b_ratio}")


  0%|          | 0/1000 [00:00<?, ?it/s]

res 0.035399112174547415
res 0.03652047508331309
res 0.03493944032771835
res 0.03289717375214526
res 0.03014927517559855
res 0.027347858740889186
res 0.02474927626355175
res 0.02265704190201238
res 0.021160494003848274
res 0.020020080716337374
res 0.019252138821717286
res 0.018658912324624258
res 0.018228318171300117
res 0.017983183988236614


  0%|          | 0/1000 [04:05<?, ?it/s]


KeyboardInterrupt: 